In [58]:
import string
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from random import randint
from pickle import load

from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [2]:
	# load doc into memory
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

In [3]:
# load document
in_filename = 'cleaned_seinfeld_scripts.txt'
doc = load_doc(in_filename)
print(doc[:2000])




                % While Jerry and Elaine are sitting on Jerry's couch watching 
                the tube, Elaine is flipping through the channels constantly.


                Jerry: What are you doing? All right, all right. What's the matter 
                with that? What about that one?


                Elaine: Robert Vaughn, The Helsinki Formula?


                Jerry: He was good in Man From Uncle.


                Elaine: Guess whose birthday's comin' up soon?


                Jerry: I know, I'm having my root canal the same week.


                Elaine: Oh, right. I hope you have a good oral surgeon because 
                that can be very serious. (Changes channel) Hey, look at naked 
                people.


                Jerry: No, I don't wanna see the naked people.


                Elaine: Been a while?


                Jerry: I have a vague recollection of doing something with someone, 
                but it was a long, long time ago.


                El

In [4]:
# turn a doc into clean tokens
def clean_doc(doc):
	doc = doc.replace('--', ' ')
	tokens = doc.split()
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	tokens = [word for word in tokens if word.isalpha()]
	tokens = [word.lower() for word in tokens]
	return tokens

In [5]:
# looking at the clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['while', 'jerry', 'and', 'elaine', 'are', 'sitting', 'on', 'jerrys', 'couch', 'watching', 'the', 'tube', 'elaine', 'is', 'flipping', 'through', 'the', 'channels', 'constantly', 'jerry', 'what', 'are', 'you', 'doing', 'all', 'right', 'all', 'right', 'whats', 'the', 'matter', 'with', 'that', 'what', 'about', 'that', 'one', 'elaine', 'robert', 'vaughn', 'the', 'helsinki', 'formula', 'jerry', 'he', 'was', 'good', 'in', 'man', 'from', 'uncle', 'elaine', 'guess', 'whose', 'birthdays', 'comin', 'up', 'soon', 'jerry', 'i', 'know', 'im', 'having', 'my', 'root', 'canal', 'the', 'same', 'week', 'elaine', 'oh', 'right', 'i', 'hope', 'you', 'have', 'a', 'good', 'oral', 'surgeon', 'because', 'that', 'can', 'be', 'very', 'serious', 'changes', 'channel', 'hey', 'look', 'at', 'naked', 'people', 'jerry', 'no', 'i', 'dont', 'wanna', 'see', 'the', 'naked', 'people', 'elaine', 'been', 'a', 'while', 'jerry', 'i', 'have', 'a', 'vague', 'recollection', 'of', 'doing', 'something', 'with', 'someone', 'but', 'i

In [6]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	seq = tokens[i-length:i]
	line = ' '.join(seq)
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 203847


In [7]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [8]:
# save sequences to file
out_filename = 'seinfeld_output.txt'
save_doc(sequences, out_filename)

In [9]:
in_filename = 'seinfeld_output.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [13]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [21]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [30]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            627850    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 12557)             1268257   
Total params: 2,047,007
Trainable params: 2,047,007
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
# training model
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
1593/1593 [==============================] - 66s 41ms/step - loss: 6.8289 - accuracy: 0.0370
Epoch 2/10
1593/1593 [==============================] - 67s 42ms/step - loss: 6.0801 - accuracy: 0.0657
Epoch 3/10
1593/1593 [==============================] - 68s 42ms/step - loss: 5.7516 - accuracy: 0.0911
Epoch 4/10
1593/1593 [==============================] - 66s 41ms/step - loss: 5.5558 - accuracy: 0.1026
Epoch 5/10
1593/1593 [==============================] - 67s 42ms/step - loss: 5.3624 - accuracy: 0.1122
Epoch 6/10
1593/1593 [==============================] - 67s 42ms/step - loss: 5.1791 - accuracy: 0.1239
Epoch 7/10
1593/1593 [==============================] - 65s 41ms/step - loss: 5.0477 - accuracy: 0.1302
Epoch 8/10
1593/1593 [==============================] - 65s 41ms/step - loss: 4.9114 - accuracy: 0.1351
Epoch 9/10
1593/1593 [==============================] - 66s 41ms/step - loss: 4.8028 - accuracy: 0.1393
Epoch 10/10
1593/1593 [==============================] - 67s 42m

In [55]:
# saving
model.save('model.h5')
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

NameError: name 'pickle' is not defined

In [56]:
# load cleaned text sequences
in_filename = 'cleaned_seinfeld_scripts.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [37]:
seq_length = len(lines[0].split()) - 1

In [50]:
# load the model
model = load_model('model.h5')

In [57]:
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'tokenizer.pkl'

In [ ]:
prompt = "[Jerry and George are on the phone with Elaine] Jerry: I'll be right there. Elaine: What happened? George:"

In [ ]:
encoded = tokenizer.texts_to_sequences([prompt])[0]

In [ ]:
def generate(model, tokenizer, seq_length, prompt, n_words):
	result = list()
	in_text = prompt
	# generate a fixed number of words
	for _ in range(n_words):
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		gen = model.predict_classes(encoded, verbose=0)

In [ ]:
generated = generate(model, tokenizer, seq_length, prompt, 100)
print(generated)